In [ ]:
import os
cwd = os.getcwd()
new_cwd = os.path.join(cwd.rsplit("SeinAcoustic_DataAnalysis")[0], "SeinAcoustic_DataAnalysis") if not cwd.endswith("SeinAcoustic_DataAnalysis") else None
if new_cwd != None:
    os.chdir(new_cwd)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import glob
from IPython.display import display

In [ ]:
from DBBuilder.datareader import Otoriver_DataReader, Sensea_DataReader
from DBBuilder import DBBuilder

In [ ]:
filepath_otoriver = "data/Data_SAM/Bougival/input/acoustique/otoriver/otoriver_20210706-20231208.csv" 
filepath_sensea = "data/Data_SAM/Bougival/input/acoustique/sensea/sensea_2021-06-16_2023-11-21.csv"
filepath_phch_continue = "data/Data_SAM/Bougival/input/phch/phch_continue/phch_continue_20210601-20230831.csv"

In [ ]:
dbbuilder = DBBuilder(output_asdataframe=True, input_timefreq="h")
output = dbbuilder.build_from_seinAcousticSourceFiles(
    acoustic_sensea=filepath_sensea, 
    acoustic_otoriver=filepath_otoriver,
    phch_continus=filepath_phch_continue)
for k, df in output["data"].items():
    print(k)
    display(df)

In [ ]:
dbbuilder.export_to_csv("data/Data_SAM/Bougival/output", baseFilename="Bougival_DataSet")

In [ ]:
filepath_db=f"data/Data_SAM/Bougival/output/Bougival_DataBase_{dbbuilder.daily_data.date.min().strftime("%Y%m%d")}-{dbbuilder.daily_data.date.max().strftime("%Y%m%d")}.xlsx"

In [ ]:
dbbuilder.export_to_xlsx(filepath=filepath_db)